# Interfaz que funciona

In [1]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Crear la ventana principal
root = tk.Tk()
root.title("Polar Plot Viewer")
root.geometry("1000x570")
root.resizable(0, 0)

# Crear el marco principal
main_frame = tk.Frame(root)
main_frame.pack(fill=tk.BOTH, expand=True)

# Crear la figura y el eje para el gráfico
figure, ax = plt.subplots(figsize=(5, 4.7),subplot_kw={'projection': 'polar'})
canvas = FigureCanvasTkAgg(figure, master=main_frame)
canvas.get_tk_widget().place(x=470, y=20)


# Función para crear la sección de monopolo
def create_monopolo_section():
    global fmonopolo_input
    monopoloSection = LabelFrame(root, width=200, height=200, text="Sección Monopolo").place(x=20, y=10)
    button = tk.Button(monopoloSection, text="Monopolo", command=plot_monopolo)
    button.place(x=30, y=170)

    fmonopoloLabel = Label(monopoloSection, text="Frecuencia [Hz]::").place(x=30, y=40)
    fmonopolo_input = tk.Entry(monopoloSection)
    fmonopolo_input.place(x=30, y=60)

# Función para crear la sección de dipolo
def create_dipolo_section():
    global b_input, f_input
    dipoloSection = LabelFrame(root, width=200, height=200, text="Sección Dipolo").place(x=240, y=10)
    button2 = tk.Button(dipoloSection, text="Dipolo", command=plot_dipolo)
    button2.place(x=250, y=170)

    b2Label = Label(dipoloSection, text="Distancia b de separación [m]:").place(x=250, y=40)
    b_input = tk.Entry(dipoloSection)
    b_input.place(x=250, y=60)

    fLabel = Label(dipoloSection, text="Frecuencia [Hz]:").place(x=250, y=80)
    f_input = tk.Entry(dipoloSection)
    f_input.place(x=250, y=100)

# Función para crear la sección de 3 fuentes
def create_fuentes_section():
    global b3_input, f3_input, patron_input, tresfuentesSection
    tresfuentesSection = LabelFrame(root, width=200, height=250, text="Sección 3 Fuentes").place(x=20, y=240)
    button3 = tk.Button(tresfuentesSection, text="3 Fuentes", command=calcular_3Fuentes)
    button3.place(x=30, y=450)

    b3Label = Label(tresfuentesSection, text="Distancia b de separación [m]:").place(x=30, y=270)
    b3_input = tk.Entry(tresfuentesSection)
    b3_input.place(x=30, y=290)

    f3Label = Label(tresfuentesSection, text="Frecuencia [Hz]:").place(x=30, y=320)
    f3_input = tk.Entry(tresfuentesSection)
    f3_input.place(x=30, y=340)

    patronLabel = Label(tresfuentesSection, text="Ingrese la combinación:").place(x=30, y=380)
    patron_input = tk.Entry(tresfuentesSection)
    patron_input.place(x=30, y=400)

# Función para crear la sección de line-array
def create_line_array_section():
    global b4_input, f4_input, linearraySection
    linearraySection = LabelFrame(root, width=200, height=250, text="Sección Line-Array").place(x=240, y=240)
    button4 = tk.Button(linearraySection, text="Line-Array", command=plot_line_array)
    button4.place(x=250, y=450)

    b4Label = Label(linearraySection, text="Distancia b de separación [m]:").place(x=250, y=270)
    b4_input = tk.Entry(linearraySection)
    b4_input.place(x=250, y=290)

    f4Label = Label(linearraySection, text='Frecuencia [Hz]:').place(x=250, y=320)
    f4_input = tk.Entry(linearraySection)
    f4_input.place(x=250, y=340)



#--------------------------------------------- Funciones de graficar y calcular ----------------------------------------------------------


def plot_monopolo():           #Esta función permite graficar el Monopolo
    frecuenciamonopolo = float(fmonopolo_input.get())
    theta_mon = np.linspace(0, 2 * np.pi, 360)
    polar_monopolo = np.ones_like(theta_mon)
    ax.clear()
    ax.plot(theta_mon, polar_monopolo, color='blue', linewidth=1.0,
           label=f'Monopolo a {frecuenciamonopolo} Hz')
    ax.set_title(f'Patrón directivo del Monopolo')
    ax.grid(alpha=0.2)
    ax.legend(loc='lower center', fontsize=8, bbox_to_anchor=(0.5, -0.14))
    canvas.draw()

def plot_dipolo():    #Esta función nos grafica el patrón directivo del Dipolo a partir de k, b, frecuencia y distancia de separación
    
    b = float(b_input.get())
    frecuencia = float(f_input.get())
    lambda_ = 343 / frecuencia  # Speed of sound in m/s / frequency
    k = (2*np.pi*frecuencia)/343
    theta = np.linspace(0, 2*np.pi, 360)
    try:
        
        dipolo = 2*1j*np.sin(k*b/2*np.sin(theta))#np.abs((2j * np.sin(((np.pi * b) / lambda_) * np.sin(theta))))

        ax.clear()
        ax.plot(theta, np.abs(dipolo), color='blue', linewidth=0.7, 
                 label=f'Dipolo a {frecuencia} Hz y a una separación de {b} m')
        ax.set_title(f'Patrón directivo del Dipolo')
        ax.set_thetagrids(range(0, 360, 30))
        ax.grid(alpha=0.2)
        ax.legend(loc='lower center', fontsize=8, bbox_to_anchor=(0.5, -0.14))
        canvas.draw()
    except ValueError:
        ax.clear()
        ax.text(0.5, 0.5, 'Valores no válidos', horizontalalignment='center', verticalalignment='center')
        ax.set_title('Error')
        canvas.draw()

def aplicar_ecuacion(patron, x):     #Esta función retorna que tipo de ecuación utilizar para arreglo de 3 fuentes según su combinación 
    
    if patron == "+++":
        return 1 + 2 * np.cos(x)
    elif patron == "---":
        return -1 - 2 * np.cos(x)
    elif patron == "+-+":
        return -1 + 2 * np.cos(x)
    elif patron == "-+-":
        return 1 - 2 * np.cos(x)
    elif patron == "++-":
        return 1 + 2j * np.sin(x)
    elif patron == "--+":
        return -1 - 2j * np.sin(x)
    else:
        return np.zeros_like(x)

def calcular_3Fuentes():          #Esta función permite graficar a partir de la combinación o patrón, k, b, frecuencia, y el vector de ángulo.
    
    patron = patron_input.get()
    error_label = Label(tresfuentesSection, text="", fg="red")
    error_label.place(x=30, y=420)
    
    try:
        patrones_validos = ["+++", "---", "+-+", "-+-", "++-", "--+"]
        if patron not in patrones_validos:
            raise ValueError("¡Patrón no válido.!")
       
        b3 = float(b3_input.get())
        frecuencia3 = float(f3_input.get())
        k3 = (2 * np.pi * frecuencia3)/343
        theta3 = np.linspace(0, 2 * np.pi, 1000)
        x3 = (k3 * b3 * np.sin(theta3))
        triArray = aplicar_ecuacion(patron, x3)

        error_label.config(text="                                ")  #desaparece mensaje de error

        ax.clear()
        ax.plot(theta3, np.abs(triArray), color='blue', linewidth=0.7, 
                 label=f'3 Fuentes en {patron} a {frecuencia3} Hz y a una separación de {b3} m')
        ax.set_title(f"Patrón de Directividad de Arreglo de 3 Fuentes")
        ax.set_thetagrids(range(0, 360, 30))
        ax.grid(alpha=0.2)
        ax.legend(loc='lower center', fontsize=8, bbox_to_anchor=(0.5, -0.14))
        canvas.draw()
    except ValueError as e:
        #print(e)
        error_label.config(text=str(e))

def plot_line_array():
  
    b4 = float(b4_input.get())
    frecuencia4 = float(f4_input.get())
    k4 = (2 * np.pi * frecuencia4)/343
    theta4 = np.linspace(0, 2 * np.pi, 1000)
    x4 = (k4 * b4 * np.sin(theta4))
    lineArray = ( ((7/8)*np.cos((3/2)*x4)) - ((1/4)*np.cos((1/2)*x4)) + (2j*np.sin((3/2)*x4)) - (((3/4)*1j)*np.sin((1/2)*x4)) )

    ax.clear()
    ax.plot(theta4, np.abs(lineArray), color='blue', linewidth=0.7, 
                 label=f'Line-Array a {frecuencia4} Hz y a una separación de {b4} m')
    ax.set_title(f"Patrón de Directividad de Line-Array")
    ax.set_thetagrids(range(0, 360, 30))
    ax.grid(alpha=0.2)
    ax.legend(loc='lower center', fontsize=8, bbox_to_anchor=(0.5, -0.14))
    canvas.draw()


def on_closing():
    plt.close('all')  # Cierra todas las figuras abiertas
    root.destroy()    # Cierra la ventana de la aplicación

# Llamar las funciones para crear las secciones
create_monopolo_section()
create_dipolo_section()
create_fuentes_section()
create_line_array_section()

# Mostrar la ventana principal
root.protocol("WM_DELETE_WINDOW", on_closing)
#plot_monopolo()  # Mostrar el gráfico inicial
root.mainloop()
